In [1]:
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import numpy as np

## Read cleaned csv

In [2]:
RouteNum = '185'


#df = pd.read_csv(f'Route_{RouteNum}_cleanCombinedCSV.csv')

df = pd.read_csv(f'{RouteNum}/{RouteNum}_cleanCombinedCSV.csv')

In [3]:
df

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION
0,2018-01-01,5971012,13,4190,76976,77166,77166,185_50,1
1,2018-01-01,5971012,14,4097,77089,77256,77268,185_50,1
2,2018-01-01,5971012,19,6139,77358,77502,77502,185_50,1
3,2018-01-01,5971012,20,4108,77425,77562,77562,185_50,1
4,2018-01-01,5971012,1,4167,76200,76251,76251,185_50,1
...,...,...,...,...,...,...,...,...,...
196938,2018-10-12,8025959,16,7294,30222,30396,30413,185_56,2
196939,2018-10-12,8025959,18,4167,30388,30520,30520,185_56,2
196940,2018-10-12,8025959,17,4156,30334,30479,30488,185_56,2
196941,2018-10-12,8025959,2,4109,29170,29175,29183,185_56,2


### Make sure the features are the correct datatypes

In [4]:
df['DAYOFSERVICE'] = pd.to_datetime(df['DAYOFSERVICE'])

In [5]:
# month
df['MONTH'] = df['DAYOFSERVICE'].dt.month

In [6]:
# Day of the week
df['WEEKDAY'] = df['DAYOFSERVICE'].dt.weekday

In [7]:
# hour
df['HOUR'] = (df['ARR/DEP_PLAN']//60)//60

### Sort dataframe

In [8]:
df.sort_values(by=['DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER'], inplace=True, ignore_index=True)

## remove all non-complete journeys
- first find what unique PROGRNUMBER each routeID has for each direction

In [9]:
# get a list of unique routeids
routeIDs = list(sorted(df['ROUTEID'].unique()))

In [10]:
# for each routeid, make it a dictionary key
# for each key, add the list of all unique stops for that routeid in the dataframe
routeIDs_dict = {}
for ident in routeIDs:
    routeIDs_dict[ident] = list(sorted(df[df['ROUTEID']==ident]['PROGRNUMBER'].unique()))

### Check that each routeid is consecutive

In [11]:
for ident in routeIDs:
    for i in range(1, len(routeIDs_dict[ident])):
        if routeIDs_dict[ident][i] - routeIDs_dict[ident][i-1] > 1:
            print(f'{ident} is not consecutive.')
            break

In [12]:
tripDays = df[(df['TRIPID']==df.shift(-1)['TRIPID']) &\
           (df['DAYOFSERVICE']==df.shift(-1)['DAYOFSERVICE']) &\
           (df['PROGRNUMBER']-df.shift(-1)['PROGRNUMBER']<-1)][['TRIPID', 'DAYOFSERVICE']]

In [13]:
tripDays.reset_index(drop=True, inplace=True)

- **if yes, move on to dropping all non-consecutive journeys from the dataframe**
- **if no, find out which PROGRNUMBER values are missing from the non-consecutive routeids**

- **for each entry in the tripDays dataframe, find all rows in the dataframe that have the same TRIPID and DAYOFSERVICE as the current row in the tripDays dataframe**
- **add the indexes of each of these rows to a list**

## If journey is non-consecutive

### x, y and z routeids are non-consecutive

- what stops are missing for the non-consecutive ROUTEIDs?

In [14]:
for ident in routeIDs:
    for i in range(1, len(routeIDs_dict[ident])):
        if routeIDs_dict[ident][i] - routeIDs_dict[ident][i-1] > 1:
            print(f'{ident}: {routeIDs_dict[ident][i]} is more than 1 away from {routeIDs_dict[ident][i-1]}')

#### We can see from this that
- ROUTEID x is missing PROGRNUMBER y
- ROUTEID a is missing PROGRNUMBER b

### Make exception cases

In [15]:
# example exception:
#exception1 = df[(df['ROUTEID']=='46A_62')&(df['PROGRNUMBER']==29)&\
                       #(df.shift(-1)['PROGRNUMBER']==31)].index

# when df routeid is x and when the current program number is one less than y AND
# the next program number is one greater than y
# add this index to the exception list.

### When all exception cases have been made , drop each of the exceptions from the tripDays dataframe made before

In [16]:
# list of indices to drop from the tripDays dataframe
#tripDayDrop = []

#for i in exception1:
    #tripDayDrop.append(tripDays[tripDays['index']==i].index[0])
    
# do this for all exceptions and add to tripDayDrop list

In [17]:
# then drop all exceptions from tripDays
#tripDays.drop(tripDays.index[tripDayDrop], inplace=True)

In [18]:
# tripDays.drop(columns=['index'], inplace=True)

In [19]:
#tripDays.reset_index(drop=True, inplace=True)

# Now we can drop incomplete trips from the main dataframe

## drop incomplete trips

In [20]:
dropIndexList = []

#for i in range(tripDays.shape[0]):
#     for debugging:
#     if i % 100 == 0:
#         print(i)
    #tripid = tripDays.loc[i, 'TRIPID']
    #date = tripDays.loc[i, 'DAYOFSERVICE']
    #dropIndex = df[(df['TRIPID']==tripid) & (df['DAYOFSERVICE']==date)].index
    #dropIndexList.append(dropIndex)

In [21]:
# make it one continuous list
#DropIndexList = []

#for i in dropIndexList:
    #for j in i:
        #DropIndexList.append(j)

In [22]:
#df.drop(df.index[DropIndexList], inplace=True)

In [23]:
#df.reset_index(drop=True, inplace=True)

### Final check for non-consecutive journeys

In [24]:
#tripDays = df[(df['TRIPID']==df.shift(-1)['TRIPID']) &\
           #(df['DAYOFSERVICE']==df.shift(-1)['DAYOFSERVICE']) &\
           #(df['PROGRNUMBER']-df.shift(-1)['PROGRNUMBER']<-1)][['TRIPID', 'DAYOFSERVICE']]

In [25]:
#tripDays

# Target feature: journey time

In [26]:
# make a dict
journeyTimes = {}
for i in range(df.shape[0]):
    journeyTimes[i] = 0

In [27]:
# loop through from 1 to the end
# the value of the index is the df current row actual arrival minus the df previous row actual departure

for i in range(0, df.shape[0]-1):
    if i % 10000 == 0:
        print(i)
    if df.loc[i, 'TRIPID'] == df.loc[i+1, 'TRIPID']:
        journeyTimes[i] = (df.loc[i+1, 'ACTUALTIME_ARR'] - df.loc[i, 'ACTUALTIME_ARR'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [28]:
# make into a dataframe
journeyTimeDF = pd.DataFrame.from_dict(journeyTimes, orient='index', columns=['JOURNEYTIME'])

In [29]:
# combine with df
df = df.join(journeyTimeDF)

## Check for negative journey times

In [30]:
df[df['JOURNEYTIME']<0]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION,MONTH,WEEKDAY,HOUR,JOURNEYTIME
1501,2018-01-03,5965686,4,4154,53445,56104,56104,185_53,1,1,2,14,-2563
1539,2018-01-03,5965688,5,4183,57113,57998,57998,185_53,1,1,2,15,-817
46103,2018-03-13,6391247,4,4154,67727,67696,67730,185_54,1,3,1,18,-363
114417,2018-06-21,7024979,4,4154,60291,60353,60401,185_51,1,6,3,16,-532
146360,2018-08-07,7326134,2,4170,35224,35248,35248,185_53,1,8,1,9,-345
146362,2018-08-07,7326134,4,4154,35384,35689,35699,185_53,1,8,1,9,-242
157911,2018-08-24,7316022,5,4183,75272,75582,75582,185_50,1,8,4,20,-64
171158,2018-09-12,8098202,5,4183,53516,55081,55081,185_53,1,9,2,14,-982


## If there negative journeytimes:

### As we can see, we have journeys with negative JOURNEYTIME times. We will find each of these entire trips and drop them all.

In [31]:
# get a list of the dates with the negative journeytimes
negDays = list(df[df['JOURNEYTIME']<0]['DAYOFSERVICE'])

In [32]:
# get a list of the trip IDs with the negative journey times
negTripIDs = list(df[df['JOURNEYTIME']<0]['TRIPID'])

In [33]:
# drop all rows that have the DAYOFSERVICE and TRIPID values at both of their respective indexes
# This means any rows that have a DAYOFSERVICE value at index 0 of the dropDays list AND a TRIPID value 
# at index 0 of the dropTripIDs list

for i in range(len(negDays)):
#     for debugging:
#     if i%10==0:
#         print(i)
    dropindex = df[(df['DAYOFSERVICE']==negDays[i])&\
                           (df['TRIPID']==negTripIDs[i])].index
    df.drop(dropindex, inplace=True)

### Final check for negative journey times

In [34]:
df[df['JOURNEYTIME']<0]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION,MONTH,WEEKDAY,HOUR,JOURNEYTIME


In [35]:
df.reset_index(drop=True, inplace=True)

## Remove outliers in terms of journey time

In [36]:
# Determine the mean, max, standard deviation as well as the (mean+2*Standard Deviation)

Upper_Value = lambda x:np.abs( x.mean()+2*x.std() )
Upper_Value.__name__ = 'Upper_Value'


aggregations = {
    'JOURNEYTIME': ['mean','max', 'min','std', Upper_Value]   
}

Stops= df.groupby(['STOPPOINTID']).agg(aggregations) 

Stops['JOURNEYTIME']

,mean,max,min,std,Upper_Value
STOPPOINTID,,,,,
2113,170.221646,793,0,73.365455,316.952555
4095,119.373079,633,0,33.565528,186.504134
4097,237.866727,1723,0,91.055736,419.978199
4103,39.780078,223,0,17.232851,74.245780
4104,57.998141,345,0,15.875068,89.748278
4105,14.128407,143,4,7.895900,29.920207
4106,38.546482,415,0,12.843834,64.234149
4108,36.428571,1504,0,45.554297,127.537165
4109,32.551081,622,16,31.702063,95.955206


In [37]:
# Create a dictionary of the stop indices and the 2std above the mean
indices, values = list(Stops.index), list(Stops['JOURNEYTIME']['Upper_Value']) 

cutoffs_Upper = {}

for ix, value in zip(indices, values):
    cutoffs_Upper[ix] = value
print(cutoffs_Upper)    

{2113: 316.9525554612544, 4095: 186.50413433278692, 4097: 419.9781990095138, 4103: 74.24578016926802, 4104: 89.748277573415, 4105: 29.920206784373, 4106: 64.23414887097294, 4108: 127.53716521835783, 4109: 95.95520601229936, 4110: 32.67955909286165, 4111: 129.48208014668364, 4112: 52.0410810770197, 4113: 127.87724722889433, 4131: 151.91153605080504, 4153: 169.39581095152994, 4154: 155.67982249919214, 4156: 467.9794193926971, 4167: 291.56952485974153, 4170: 258.13720569070097, 4183: 682.5271768967071, 4184: 62.40588488180776, 4185: 91.77869032824594, 4186: 99.61503430078184, 4187: 64.41010496703689, 4188: 297.55223007277425, 4189: 155.14785016575786, 4190: 244.3445784412793, 4192: 250.0241445074576, 4194: 155.15365730617327, 4196: 76.6666934457109, 4199: 236.13009901378024, 4416: 311.07202110634427, 4417: 132.61065577648316, 4419: 157.31269594297987, 4420: 113.17576551869664, 4421: 348.8431997384472, 4422: 88.46326921699668, 4584: 76.0042817075769, 6139: 430.19672078803325, 7294: 205.677

In [38]:
df

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION,MONTH,WEEKDAY,HOUR,JOURNEYTIME
0,2018-01-01,5970996,1,4167,37800,37839,37839,185_55,1,1,0,10,58
1,2018-01-01,5970996,2,4170,37890,37897,37897,185_55,1,1,0,10,89
2,2018-01-01,5970996,3,4153,37950,37986,37986,185_55,1,1,0,10,26
3,2018-01-01,5970996,4,4154,38005,38012,38023,185_55,1,1,0,10,69
4,2018-01-01,5970996,5,4416,38051,38081,38081,185_55,1,1,0,10,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196806,2018-10-20,8030188,5,4183,61440,62062,62062,185_52,1,10,5,17,138
196807,2018-10-20,8030188,13,4417,61981,62200,62211,185_52,1,10,5,17,32
196808,2018-10-20,8030188,14,2113,61984,62232,62232,185_52,1,10,5,17,32
196809,2018-10-20,8030188,15,4419,62008,62264,62275,185_52,1,10,5,17,74


In [39]:
df['Cut_Off_Upper'] = df[['STOPPOINTID', 'JOURNEYTIME']].apply(lambda x: x[1] < cutoffs_Upper[x[0]], axis = 1)

### remove outliers in terms of the standard deviation

In [40]:
# Remove the outliers for the 2std above the mean
df.drop(df.loc[df['Cut_Off_Upper']== False].index, inplace=True)

In [41]:
df.reset_index(inplace=True, drop=True)

## add a feature: end stop
- and rename STOPPOINTID to startStop

In [42]:
df.rename(columns={'STOPPOINTID':'startStop'}, inplace=True)

In [43]:
df['endStop'] = 'N/A'

In [44]:
df.loc[(df['TRIPID']==df['TRIPID'].shift(-1)), 'endStop'] = df['startStop'].shift(-1)

In [45]:
df

,DAYOFSERVICE,TRIPID,PROGRNUMBER,startStop,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION,MONTH,WEEKDAY,HOUR,JOURNEYTIME,Cut_Off_Upper,endStop
0,2018-01-01,5970996,1,4167,37800,37839,37839,185_55,1,1,0,10,58,True,4170.0
1,2018-01-01,5970996,2,4170,37890,37897,37897,185_55,1,1,0,10,89,True,4153.0
2,2018-01-01,5970996,3,4153,37950,37986,37986,185_55,1,1,0,10,26,True,4154.0
3,2018-01-01,5970996,4,4154,38005,38012,38023,185_55,1,1,0,10,69,True,4416.0
4,2018-01-01,5970996,5,4416,38051,38081,38081,185_55,1,1,0,10,63,True,4417.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193012,2018-10-20,8030188,5,4183,61440,62062,62062,185_52,1,10,5,17,138,True,4417.0
193013,2018-10-20,8030188,13,4417,61981,62200,62211,185_52,1,10,5,17,32,True,2113.0
193014,2018-10-20,8030188,14,2113,61984,62232,62232,185_52,1,10,5,17,32,True,4419.0
193015,2018-10-20,8030188,15,4419,62008,62264,62275,185_52,1,10,5,17,74,True,4420.0


## Remove all rows that have an endStop value of 'N/A'

In [46]:
dropIndex = list(df[df['endStop']=='N/A'].index)

In [47]:
df.drop(dropIndex, inplace=True)

In [48]:
df.reset_index(drop=True, inplace=True)

## Merge with weather dataframe
- create DAYHOUR to combine two dataframes on

In [49]:
df['DAYHOUR'] = df['DAYOFSERVICE'] + pd.to_timedelta(df['HOUR'], unit='H')

In [50]:
weather = pd.read_csv('weatherData_CLEANED.csv')
weather['DAYHOUR'] = pd.to_datetime(weather['DAYHOUR'])

- drop descriptive weather features

In [51]:
weather.reset_index(drop=True, inplace=True)

In [52]:
weather.drop(columns=['weather_main', 'weather_description'], inplace=True)

- add precipitation feature

In [53]:
# change nan values to 0 for rain and snow
nullRain = weather.loc[pd.isnull(weather['rain_1h']), 'rain_1h'].index
weather.loc[nullRain, 'rain_1h'] = 0

nullSnow = weather.loc[pd.isnull(weather['snow_1h']), 'snow_1h'].index
weather.loc[nullSnow, 'snow_1h'] = 0

In [54]:
weather['precipitation_1h'] = ((weather['rain_1h']) + (weather['snow_1h']))

In [55]:
weather.drop(columns=['rain_1h', 'snow_1h'], inplace=True)

### Change precipitation values that are NaN to 0

In [56]:
df = pd.merge(df, weather, on=['DAYHOUR'])
df.drop(columns=['DAYHOUR'])

,DAYOFSERVICE,TRIPID,PROGRNUMBER,startStop,ARR/DEP_PLAN,ACTUALTIME_ARR,ACTUALTIME_DEP,ROUTEID,DIRECTION,MONTH,WEEKDAY,HOUR,JOURNEYTIME,Cut_Off_Upper,endStop,temp,humidity,wind_speed,precipitation_1h
0,2018-01-01,5970996,1,4167,37800,37839,37839,185_55,1,1,0,10,58,True,4170.0,5.02,81,9.77,0.0
1,2018-01-01,5970996,2,4170,37890,37897,37897,185_55,1,1,0,10,89,True,4153.0,5.02,81,9.77,0.0
2,2018-01-01,5970996,3,4153,37950,37986,37986,185_55,1,1,0,10,26,True,4154.0,5.02,81,9.77,0.0
3,2018-01-01,5970996,4,4154,38005,38012,38023,185_55,1,1,0,10,69,True,4416.0,5.02,81,9.77,0.0
4,2018-01-01,5970996,5,4416,38051,38081,38081,185_55,1,1,0,10,63,True,4417.0,5.02,81,9.77,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180036,2018-10-20,8019803,10,4196,85975,85851,85851,185_63,2,10,5,23,24,True,4422.0,14.69,82,3.09,0.0
180037,2018-10-20,8019803,11,4422,85999,85875,85875,185_63,2,10,5,23,36,True,7369.0,14.69,82,3.09,0.0
180038,2018-10-20,8019803,12,7369,86051,85911,85911,185_63,2,10,5,23,14,True,7295.0,14.69,82,3.09,0.0
180039,2018-10-20,8019803,13,7295,86077,85925,85925,185_63,2,10,5,23,44,True,4199.0,14.69,82,3.09,0.0


In [57]:
df = df[['DIRECTION', 'MONTH', 'WEEKDAY', 'HOUR', 'startStop', 'endStop', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'JOURNEYTIME',\
        'temp', 'humidity', 'wind_speed', 'precipitation_1h']]

## Save df as csv file

In [58]:
df

,DIRECTION,MONTH,WEEKDAY,HOUR,startStop,endStop,ACTUALTIME_ARR,ACTUALTIME_DEP,JOURNEYTIME,temp,humidity,wind_speed,precipitation_1h
0,1,1,0,10,4167,4170.0,37839,37839,58,5.02,81,9.77,0.0
1,1,1,0,10,4170,4153.0,37897,37897,89,5.02,81,9.77,0.0
2,1,1,0,10,4153,4154.0,37986,37986,26,5.02,81,9.77,0.0
3,1,1,0,10,4154,4416.0,38012,38023,69,5.02,81,9.77,0.0
4,1,1,0,10,4416,4417.0,38081,38081,63,5.02,81,9.77,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180036,2,10,5,23,4196,4422.0,85851,85851,24,14.69,82,3.09,0.0
180037,2,10,5,23,4422,7369.0,85875,85875,36,14.69,82,3.09,0.0
180038,2,10,5,23,7369,7295.0,85911,85911,14,14.69,82,3.09,0.0
180039,2,10,5,23,7295,4199.0,85925,85925,44,14.69,82,3.09,0.0


In [59]:
df.to_csv(f'{RouteNum}/{RouteNum}_MODELING.csv', index_label=False)